# Projet 4 : Anticiper les besoins en consommation électrique de bâtiments
*Pierre-Eloi Ragetly*

Ce projet fait parti du parcours *DataScientist* d'OpenClassrooms.

L'objectif principal est de trouver un modèle permettant de prédire **les émissions de CO2 et la consommation totale d’énergie de bâtiments non destinés à l'habitation.**

Pour cela nous disposons des données de la ville de Seattle pour les années 2015 et 2016. Ces données sont à récupérer sur le site kaggle.

# Partie III : Data modeling

Ce notebook a pour but de présenter le travail effectué sur la modélisation.

Dans un premier temps les modèles les plus courants seront entraînés et testés (une séparation des données sera faite au préalable) sur les données de 2015. Les modèles plus prometteurs seront conservés. Vu qu'il s'agit d'un problème de regression nous utiliserons la **RMSE** (Root Mean Square Error) comme mesure de performance.

Puis nous améliorerons les performances de nos modèles en ne gardant que les variables significatives et en affinant les hyperparamètres à l'aide d'une recherche par quadrillage (nous testerons aussi une recherche aléatoire.)

La dernière étape de la partie modélisation consistera à combiner les meilleurs modèles afin d'obtenir un modèle encore plus performant.

Enfin, le modèle final sera testé sur le jeu de données de 2016 nettoyé afin d'évaluer sa performance.

In [1]:
# Import des librairies usuelles
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
import pandas as pd
import seaborn as sns
import os

In [2]:
# Change some default parameters of matplotlib using seaborn
plt.rcParams.update(plt.rcParamsDefault)
plt.rcParams.update({'axes.titleweight': 'bold'})
sns.set(style='ticks')
current_palette = sns.color_palette('RdBu')
sns.set_palette(current_palette)

In [3]:
df_train = (pd.read_csv('data/data_2015_tr.csv')
              .set_index('OSEBuildingID')
              .drop(columns='ENERGYSTARScore'))

df_test = pd.read_csv('data/data_2016_tr.csv').set_index('OSEBuildingID')

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Résultats-des-modèles-les-plus-courants-pour-la-consommation" data-toc-modified-id="Résultats-des-modèles-les-plus-courants-pour-la-consommation-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Résultats des modèles les plus courants pour la consommation</a></span></li></ul></div>

## Résultats des modèles les plus courants pour la consommation

Pour comparer les résultats obtenus avec les algorithmes de regression les plus courants (voir liste ci-dessous), nous utiliserons la librairie *Scikit-Learn* ainsi que la librairie *XGBoost*.
- Régression Ridge
- Régression Lasso
- Elastic Net
- Regression SVM
- Arbre de décision
- Forêt aléatoire
- Gradient Boosting
- XG Boost
- Perceptron

Nous utiliserons comme valeur cible le log de la consommation normalisée, et comme mesure de performance la RMSE par validation croisée.

In [29]:
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.svm import LinearSVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
#from xgboost import XGBRegressor
from sklearn.linear_model import Perceptron
from sklearn.model_selection import cross_val_score

def compare_models(X, y):
    """ Fonction to compare the RMSE get with the
    most common Machine Learning Regressors.
    -----------
    Parameters:
    X: Array
        the array object holding data
    y: Array
        the target
    -----------
    Return:
        DataFrame
    """
    # Create models
    ridge = Ridge()
    lasso = Lasso()
    elastic_net = ElasticNet()
    svm_reg = LinearSVR()
    tree_reg = DecisionTreeRegressor()
    forest_reg = RandomForestRegressor()
    gbrt = GradientBoostingRegressor()
    # xgbr = XGBRegressor()
    perceptron = Perceptron()
    models = [ridge,
              lasso,
              elastic_net,
              #svm_reg,
              tree_reg,
              forest_reg,
              gbrt]
    # Train models
    scores = []
    names = []
    std_rmse = []
    for m in models:
        m.fit(X, y)
        m_scores = cross_val_score(m,X, y,
                                   scoring="neg_mean_squared_error",
                                   cv=10)
        m_scores = np.sqrt(-m_scores)
        m_names = type(m).__name__
        scores.append(m_scores.mean())
        std_rmse.append(m_scores.std())
        names.append(m_names)
    # Create the DataFrame
    df = pd.DataFrame({'RMSE_mean': scores, 'RMSE_std': std_rmse}, index=names)
    return df

In [30]:
X = df_train.iloc[:, :-6].values
y = df_train.iloc[:, -1].values
df_comp =  compare_models(X, y)
df_comp

,RMSE_mean,RMSE_std
Ridge,0.742024,0.083778
Lasso,1.318409,0.160921
ElasticNet,1.132786,0.157217
DecisionTreeRegressor,0.321335,0.047397
RandomForestRegressor,0.229443,0.027190
GradientBoostingRegressor,0.212370,0.024227
